# UTS Deep Learning Number 1
Name: Austin Kane\
NIM: 270222932\
Dataset: [Dataset B](https://drive.google.com/drive/folders/1pThuel6dU23CLRseKzymT4oNDUECq_c6)\
Video Link: [Video1]()

# 1. Artificial Neural Network
Anda adalah seorang Data Scientist di perusahaan komersil. Tugas kalian adalah membuat model **Artificial Neural Network (ANN)** yang dapat menilai produktivitas setiap team dari perusahaan produsen pakaian.
Download dataset yang telah disediakan dan kerjakan task berikut ini.
1. [LO2 – 5 Poin] Lakukan **Exploratory Data Analysis (EDA)** untuk memahami kondisi data. Jelaskan semua masalah yang anda temukan pada dataset anda. Lakukan pre-processing pada dataset anda sesuai dengan hasil EDA anda, termasuk memisahkan dataset anda menjadi train, val, dan test dengan proporsi 70:10:20.
2. [LO2, LO3, & LO4 – 10 Poin] Buatlah **2 baseline model** dengan jumlah layer dan neuron yang berbeda. 1 model berupa **Sequential Model** dan 1 lagi berupa **Functional Model**. Semua hidden layer wajib menggunakan activation function bernama ReLU dan memiliki jumlah neuron minimal 2 kali lipat dari dimensi input data. Kedua model harus memiliki minimal 2 hidden layer. Lakukan training pada kedua model tersebut dengan minimal 10 epoch.
3. [LO2, LO3, & LO4 – 10 Poin] Lakukan **modifikasi pada kedua model anda**. Anda dapat mengubah jumlah neuron dan layer ataupun activation function dari hidden layer. Anda juga dapat melakukan **hyperparameter fine-tuning** pada model anda. Lakukan training pada 2 modifikasi model anda.
4. [LO2, LO3, & LO4 – 10 Poin] Lakukan **evaluasi pada 4 model** yang sudah anda buat menggunakan **minimal 3 evaluation metrics**, lalu bandingkan, analisis, dan simpulkan hasilnya.
5. [LO1, LO2 – 5 Poin] Buatlah **video presentasi** yang menjelaskan langkah-langkah pengerjaan anda serta hasil analisis anda dengan durasi maksimal 15 menit.

## Import Libraries

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Read the Data

In [13]:
data = pd.read_parquet("Data/01/dataset_1B.parquet")
data.head()

,date,quarter,day,Team Code,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,productivity_score
0,1/1/2015,Quarter1,Thursday,8,26.16,1108.0,7080,98,0.0,0,0,59.0,94.073
1,1/1/2015,Quarter1,Thursday,1,3.94,NaN,960,0,0.0,0,0,8.0,88.650
2,1/1/2015,Quarter1,Thursday,11,11.41,968.0,3660,50,0.0,0,0,30.5,80.057
3,1/1/2015,Quarter1,Thursday,12,11.41,968.0,3660,50,0.0,0,0,30.5,80.057
4,1/1/2015,Quarter1,Thursday,6,25.90,1170.0,1920,50,0.0,0,0,56.0,80.038


## Explaratory Data Analysis (EDA)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                1197 non-null   object 
 1   quarter             1197 non-null   object 
 2   day                 1197 non-null   object 
 3   Team Code           1197 non-null   int64  
 4   smv                 1197 non-null   float64
 5   wip                 691 non-null    float64
 6   over_time           1197 non-null   int64  
 7   incentive           1197 non-null   int64  
 8   idle_time           1197 non-null   float64
 9   idle_men            1197 non-null   int64  
 10  no_of_style_change  1197 non-null   int64  
 11  no_of_workers       1197 non-null   float64
 12  productivity_score  1197 non-null   float64
dtypes: float64(5), int64(5), object(3)
memory usage: 121.7+ KB


In [15]:
data.describe()

,Team Code,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,productivity_score
count,1197.000000,1197.000000,691.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000
mean,6.426901,15.062172,1190.465991,4567.460317,38.210526,0.730159,0.369256,0.150376,34.337510,73.367040
std,3.463963,10.943219,1837.455001,3348.823563,160.182643,12.709757,3.268987,0.427848,22.617043,18.154945
min,1.000000,2.900000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-57.000000,-100.000000
25%,3.000000,3.940000,774.500000,1440.000000,0.000000,0.000000,0.000000,0.000000,9.000000,65.030000
50%,6.000000,15.260000,1039.000000,3960.000000,0.000000,0.000000,0.000000,0.000000,34.000000,77.333000
75%,9.000000,24.260000,1252.500000,6960.000000,50.000000,0.000000,0.000000,0.000000,57.000000,85.025000
max,12.000000,54.560000,23122.000000,25920.000000,3600.000000,300.000000,45.000000,2.000000,89.000000,112.044000


In [19]:
data.isna().sum()

date                    0
quarter                 0
day                     0
Team Code               0
smv                     0
wip                   506
over_time               0
incentive               0
idle_time               0
idle_men                0
no_of_style_change      0
no_of_workers           0
productivity_score      0
dtype: int64

In [20]:
data.duplicated().sum()

np.int64(0)

In [24]:
for col in data.columns:
    print(f"{col}: {data[col].nunique()}")

date: 118
quarter: 5
day: 6
Team Code: 12
smv: 70
wip: 548
over_time: 143
incentive: 48
idle_time: 12
idle_men: 10
no_of_style_change: 3
no_of_workers: 66
productivity_score: 803


In [27]:
for col in data.columns:
    if data[col].nunique()<80:
        print(f"{data[col].value_counts()}\n")

quarter
Quarter1    360
Quarter2    335
Quarter4    248
Quarter3    210
Quarter5     44
Name: count, dtype: int64

day
Wednesday    208
Sunday       203
Tuesday      201
Thursday     199
Monday       199
Saturday     187
Name: count, dtype: int64

Team Code
8     109
2     109
4     105
1     105
9     104
10    100
12     99
7      96
3      95
6      94
5      93
11     88
Name: count, dtype: int64

smv
3.94     192
2.90     108
22.52    103
30.10     79
4.15      76
        ... 
38.09      1
48.18      1
30.40      1
50.89      1
20.20      1
Name: count, Length: 70, dtype: int64

incentive
0       604
50      113
63       61
45       54
30       52
23       38
38       29
60       28
40       27
75       24
113      21
88       19
34       17
56       14
26        9
55        7
81        7
100       7
65        6
69        6
70        6
960       5
35        5
44        4
94        4
90        3
49        2
27        2
46        2
119       2
24        2
98        1
29        1
54 

Insights:
- abc

## Data Preprocessing

To-do:
- abc

## Baseline Model

## Hyperparameter Fine-Tuning

## Evaluation